#SoMe Topic Modeling Notebook | Release canvas 1 📖

## Installations and Libraries 💽

In [73]:
# Installations
import sys
if 'google.colab' in sys.modules:
    !pip install emoji --upgrade
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis
    !pip install gensim
    !pip install squarify==0.4.3

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.5.4)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
import squarify
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib as plt 
import pyLDAvis.gensim

#Natural Language Processing (NLP)
import spacy
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

##Data Cleaning 🧹

In [95]:
# Loading the JSON file 
url_elon = 'https://raw.githubusercontent.com/Lambda-School-Labs/social-media-strategy-ds/feature/topic/python_notebooks/elonmusk_followers_english.json'
url_dutchbros = 'https://raw.githubusercontent.com/Lambda-School-Labs/social-media-strategy-ds/feature/topic/python_notebooks/dutchbros_followers.json'

df = requests.get(url_elon).json()

# Converting the dataset to pandas DataFrame and renaming the columns 
df = pd.DataFrame(df.values())
df = df.rename(columns={0:'original_tweets'})

#Removing emojies from text and and puttin them in a new column
#Refrence 1 : https://stackoverflow.com/questions/43146528/how-to-extract-all-the-emojis-from-text
#Refrence 2 : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
#Refrence 3 : https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python

### Question: How to combine all functions into one 
def extract_emoji(text):
    '''
    Extracts emojies from text
    '''
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    return emoji_list

def emoji_free_text(text):
    '''
    Cleans text from emojies
    '''
    emoji_list_1 = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list_1)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

#Create a new column with emojies from each tweet
df['tweet_emojies'] = df['original_tweets'].apply(extract_emoji)
### .apply extract emoji 

#Create a new column with cleaned tweets from emojies
df['emoji_free_tweets'] = df['original_tweets'].apply(emoji_free_text)
### .apply extract emoji 

#Create a new column with url free tweets
df['url_free_tweets'] = df['emoji_free_tweets'].apply(url_free_text)
### .apply extract emoji 
df

,original_tweets,tweet_emojies,emoji_free_tweets,url_free_tweets
0,This kid will forever be a legend 😂 https://t....,[😂],This kid will forever be a legend https://t.co...,This kid will forever be a legend
1,"If you truly believe Lebrons mindset, competit...",[],"If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit..."
2,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,[],BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!
3,@Bhuvan_Bam ❤️❤️,"[❤️, ❤️]",@Bhuvan_Bam,@Bhuvan_Bam
4,I'm not crying you're crying.\nhttps://t.co/Bc...,[],I'm not crying you're crying. https://t.co/BcF...,I'm not crying you're crying.
...,...,...,...,...
9941,@MirandaSleeper The offerings in that f-3 are ...,[],@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...
9942,This will be the defining segment of “The Last...,[],This will be the defining segment of “The Last...,This will be the defining segment of “The Last...
9943,"@frank_miskelly I don’t like it, I LOVE IT! Bu...",[],"@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu..."
9944,Excited for my brother @Shufly10 as he embarks...,[],Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...


##Topic Modeling ㊙️

###Tokenizing 🕵🏻‍♂

In [0]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
nlp = spacy.load('en_core_web_lg')

In [97]:
"""
Import Gensim and Wordcloud to use their stopwords as well and use the combined stopwords of ALL as the variable:
ALL_STOP_WORDS
"""

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


tokens = []

for doc in tokenizer.pipe(df['url_free_tweets'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
df['tokens'] = tokens

# View df
df

,original_tweets,tweet_emojies,emoji_free_tweets,url_free_tweets,tokens
0,This kid will forever be a legend 😂 https://t....,[😂],This kid will forever be a legend https://t.co...,This kid will forever be a legend,"[kid, forever, legend]"
1,"If you truly believe Lebrons mindset, competit...",[],"If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit...","[truly, believe, lebrons, mindset,, competitiv..."
2,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,[],BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,"[buttlicker!, prices, lower!!!]"
3,@Bhuvan_Bam ❤️❤️,"[❤️, ❤️]",@Bhuvan_Bam,@Bhuvan_Bam,[@bhuvan_bam]
4,I'm not crying you're crying.\nhttps://t.co/Bc...,[],I'm not crying you're crying. https://t.co/BcF...,I'm not crying you're crying.,"[crying, you're, crying.]"
...,...,...,...,...,...
9941,@MirandaSleeper The offerings in that f-3 are ...,[],@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...,"[@mirandasleeper, offerings, f-3, closed, prev..."
9942,This will be the defining segment of “The Last...,[],This will be the defining segment of “The Last...,This will be the defining segment of “The Last...,"[defining, segment, “the, dance.”]"
9943,"@frank_miskelly I don’t like it, I LOVE IT! Bu...",[],"@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu...","[@frank_miskelly, don’t, it,, love, it!, think..."
9944,Excited for my brother @Shufly10 as he embarks...,[],Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...,"[excited, brother, @shufly10, embarks, new, jo..."


###Lemmatization🇬🇧

In [0]:
# Make tokens a string again
# Refrence 4 : https://stackoverflow.com/questions/45306988/column-of-lists-convert-list-to-string-as-a-new-column
df['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

df['lemmas'] = df['tokens_back_to_text'].apply(get_lemmas)

# Make lemmas a string again
df['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in df['lemmas']]
# df[['original_tweet', 'lemmas_back_to_text']]

In [99]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(pattern, "", text) # https://www.youtube.com/watch?v=O2onA4r5UaY
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub('@*!*\$*', '', text) # Remove @ ! $
    #tokens = re.sub('@', '', text) # Remove @
    #tokens = re.sub('$', '', text) # Remove $
    #tokens = re.sub('\$*', '', text) # Remove $
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE
    #tokens = tokens.strip("$") # TESTING THIS LINE
    #tokens = tokens.strip("@") # TESTING THIS LINE

    tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
df['lemma_tokens'] = df['lemmas_back_to_text'].apply(tokenize)

# View those tokens (the 4th column)
df

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationWarning:

invalid escape sequence \$

<input>:19: DeprecationWarning:

invalid escape sequence \w

<input>:20: DeprecationW

,original_tweets,tweet_emojies,emoji_free_tweets,url_free_tweets,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,This kid will forever be a legend 😂 https://t....,[😂],This kid will forever be a legend https://t.co...,This kid will forever be a legend,"[kid, forever, legend]",kid forever legend,"[kid, forever, legend]",kid forever legend,"[kid, forever, legend]"
1,"If you truly believe Lebrons mindset, competit...",[],"If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit...","[truly, believe, lebrons, mindset,, competitiv...","truly believe lebrons mindset, competitive fir...","[truly, believe, lebrons, mindset, competitive...",truly believe lebrons mindset competitive fire...,"[truly, believe, lebrons, mindset, competitive..."
2,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,[],BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,"[buttlicker!, prices, lower!!!]",buttlicker! prices lower!!!,"[buttlicker, price, lower]",buttlicker price lower,"[buttlicker, price, lower]"
3,@Bhuvan_Bam ❤️❤️,"[❤️, ❤️]",@Bhuvan_Bam,@Bhuvan_Bam,[@bhuvan_bam],@bhuvan_bam,[@bhuvan_bam],@bhuvan_bam,[bhuvan_bam]
4,I'm not crying you're crying.\nhttps://t.co/Bc...,[],I'm not crying you're crying. https://t.co/BcF...,I'm not crying you're crying.,"[crying, you're, crying.]",crying you're crying.,"[cry, cry]",cry cry,"[cry, cry]"
...,...,...,...,...,...,...,...,...,...
9941,@MirandaSleeper The offerings in that f-3 are ...,[],@MirandaSleeper The offerings in that f-3 are ...,@MirandaSleeper The offerings in that f-3 are ...,"[@mirandasleeper, offerings, f-3, closed, prev...",@mirandasleeper offerings f-3 closed previous ...,"[@mirandasleeper, offering, f-3, close, previo...",@mirandasleeper offering f-3 close previous of...,"[mirandasleeper, offering, f-3, close, previou..."
9942,This will be the defining segment of “The Last...,[],This will be the defining segment of “The Last...,This will be the defining segment of “The Last...,"[defining, segment, “the, dance.”]",defining segment “the dance.”,"[define, segment, dance]",define segment dance,"[define, segment, dance]"
9943,"@frank_miskelly I don’t like it, I LOVE IT! Bu...",[],"@frank_miskelly I don’t like it, I LOVE IT! Bu...","@frank_miskelly I don’t like it, I LOVE IT! Bu...","[@frank_miskelly, don’t, it,, love, it!, think...","@frank_miskelly don’t it, love it! think sixth...","[@frank_miskelly, love, think, sixth, old, run...",@frank_miskelly love think sixth old run middl...,"[frank_miskelly, love, think, sixth, old, run,..."
9944,Excited for my brother @Shufly10 as he embarks...,[],Excited for my brother @Shufly10 as he embarks...,Excited for my brother @Shufly10 as he embarks...,"[excited, brother, @shufly10, embarks, new, jo...","excited brother @shufly10 embarks new journey,...","[excited, brother, @shufly10, embark, new, jou...",excited brother @shufly10 embark new journey o...,"[excited, brother, shufly10, embark, new, jour..."


###id2word 📒

In [100]:
# Create a id2word dictionary
id2word = Dictionary(df['lemma_tokens'])
print(len(id2word))

21883


In [101]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

8229


###Corpus Object & Generating Topics 📚

In [0]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df['lemma_tokens']]

In [103]:
# Instantiating a LDA model 
model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)

Streaming output truncated to the last 5000 lines.

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.6/dist-pa

In [0]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model.print_topics()]

In [0]:
# Create Topics
topics = [' '.join(t[0:10]) for t in words]

In [106]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
time people need tweet follow new 2 love today good

------ Topic 1 ------
say obamagate trump time people new big win thing 1

------ Topic 2 ------
people bitcoin go good think work follow btc come new

------ Topic 3 ------
know day time think look mother thank year realdonaldtrump see

------ Topic 4 ------
know trump people obama day good flynn realdonaldtrump president state



#### Topic Distance Visualization 📈

In [107]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.053958 -0.014915       1        1  25.713909
4      0.020870  0.001267       2        1  22.574497
0     -0.067397 -0.043367       3        1  19.814075
1      0.092816 -0.049225       4        1  16.371145
3      0.007669  0.106240       5        1  15.526374, topic_info=          Term        Freq       Total Category  logprob  loglift
36        know  435.000000  435.000000  Default  30.0000  30.0000
565    bitcoin  289.000000  289.000000  Default  29.0000  29.0000
802  obamagate  182.000000  182.000000  Default  28.0000  28.0000
891      obama  183.000000  183.000000  Default  27.0000  27.0000
63         day  423.000000  423.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
144       good   46.340446  434.815186   Topic5  -5.7564  -0.3763
11        love   38.530685  265.611084   Topic5  -5.9410  -0.0679
337      start   33.583408  158.855042   Topic5  -6.0784   0.3087
361     people   42.406731  615.724915   Topic5  -5.8451  -0.8129
61         try   33.234058  182.063614   Topic5  -6.0888   0.1618

[446 rows x 6 columns], token_table=      Topic      Freq Term
term                      
784       1  0.540724    +
784       2  0.078651    +
784       3  0.176964    +
784       4  0.049157    +
784       5  0.157302    +
...     ...       ...  ...
7431      4  0.775872  ہیں
7411      4  0.951103   ہے
1596      5  0.827817    ᵕ
4552      3  0.928976    ★
4552      4  0.077415    ★

[1096 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 2, 4])

##Make a pipeline 🏭

In [108]:
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline

transformer = make_column_transformer()

pipelinne = make_pipeline(
    
)

ValueError: ignored